### 1.  OBJETIVO DO CÓDIGO

Calcular automaticamente quanto você precisa repor de cada produto em cada loja, semana a semana, para nunca faltar estoque.

In [ ]:
#ativar venv --> .venv\Scripts\activate

from pathlib import Path
import pandas as pd
import numpy as np


DATA_DIR = Path("data")
SAIDA_DIR = Path("outputs")
SAIDA_DIR.mkdir(exist_ok=True)

ARQUIVO_VENDAS = "forecast.xlsx"
ABA_VENDAS = "BD"
ARQUIVO_ESTOQUE = DATA_DIR / "estoque-total-semana-38-RA.xlsx"
ARQUIVO_CARTEIRA = DATA_DIR / "carteira.xlsx"

SEMANA_INICIO = "W39_25"
SEMANAS_ALVO = 29  # Cobertura do cenário 1 até 30/03/2026


In [ ]:
# ==================================================
# 1. FUNÇÃO: CARREGAR VENDAS (AGORA POR SKU/PDV)
# ==================================================
def carregar_vendas(caminho_arquivo, nome_aba):
    """Carrega e processa arquivo de vendas por SKU/PDV"""
    print("Carregando vendas por SKU/PDV...")
    
    df = pd.read_excel(caminho_arquivo, sheet_name=nome_aba, engine="openpyxl")
    df.columns = df.columns.str.strip()
    
    df["FILIAL"] = (
        df["FILIAL"]
        .astype(str)
        .str.strip()
    )
    df["FILIAL"] = df["FILIAL"].str.extract(r"(\d+)")[0].fillna("0")
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)
    
    for col in ["PRODUTO", "COR", "TAMANHO", "SKU"]:
        df[col] = df[col].astype(str).str.upper().str.strip()
    
    semanas_colunas = [col for col in df.columns if col.startswith("202") and "_W" in col]
    
    rename_map = {}
    for col in semanas_colunas:
        ano, semana = col.split("_W")
        semana = semana.zfill(2)
        rename_map[col] = f"W{semana}_{ano[-2:]}"
    
    df = df.rename(columns=rename_map)
    
    def sort_key(semana_rotulo):
        sem, ano = semana_rotulo.split("_")
        return int(ano), int(sem[1:])
    
    semanas = sorted(rename_map.values(), key=sort_key)
    
    colunas_util = ["FILIAL", "SKU", "PRODUTO", "COR", "TAMANHO"] + semanas
    df_util = df[colunas_util]
    
    agg_dict = {sem: "sum" for sem in semanas}
    agg_dict.update({"PRODUTO": "first", "COR": "first", "TAMANHO": "first"})
    
    df_agrupado = df_util.groupby(["FILIAL", "SKU"], as_index=False).agg(agg_dict)
    
    print(f"[OK] Vendas: {len(df_agrupado)} SKU/PDV, {len(semanas)} semanas")
    return df_agrupado, semanas


In [ ]:
# ==================================================
# 2. FUNÇÃO: CARREGAR ESTOQUE E CARTEIRA
# ==================================================
def carregar_estoque(caminho_arquivo):
    """Carrega e consolida estoque atual por SKU/PDV"""
    print("Carregando estoque atual por SKU/PDV...")
    df = pd.read_excel(caminho_arquivo, engine="openpyxl")
    df.columns = df.columns.str.strip()
    
    for candidato in ["Ponto Venda Cód", "Ponto Venda Cod", "FILIAL"]:
        if candidato in df.columns:
            df = df.rename(columns={candidato: "FILIAL"})
            break
    
    if "Estoque Total" in df.columns:
        df = df.rename(columns={"Estoque Total": "ESTOQUE_ATUAL"})
    
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)
    df["SKU"] = df["SKU"].astype(str).str.upper().str.strip()
    df["ESTOQUE_ATUAL"] = pd.to_numeric(df["ESTOQUE_ATUAL"], errors="coerce").fillna(0)
    
    df = df.groupby(["FILIAL", "SKU"], as_index=False)["ESTOQUE_ATUAL"].sum()
    print(f"[OK] Estoque consolidado: {len(df)} SKU/PDV")
    return df


def carregar_carteira(caminho_arquivo):
    """Carrega e consolida carteira de pedidos por SKU/PDV"""
    print("Carregando carteira por SKU/PDV...")
    df = pd.read_excel(caminho_arquivo, engine="openpyxl")
    df.columns = df.columns.str.strip()
    
    if "Filial" in df.columns:
        df = df.rename(columns={"Filial": "FILIAL"})
    if "Pecas" in df.columns:
        df = df.rename(columns={"Pecas": "CARTEIRA_TOTAL"})
    
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)
    df["SKU"] = df["SKU"].astype(str).str.upper().str.strip()
    df["CARTEIRA_TOTAL"] = pd.to_numeric(df["CARTEIRA_TOTAL"], errors="coerce").fillna(0)
    
    df = df.groupby(["FILIAL", "SKU"], as_index=False)["CARTEIRA_TOTAL"].sum()
    print(f"[OK] Carteira consolidada: {len(df)} SKU/PDV")
    return df


In [ ]:
# ==================================================
# 3. FUNÇÃO: CALCULAR ALVOS DE ESTOQUE
#  ALVO DE ESTOQUE = Quantidade ideal que você quer ter para cobrir as próximas X semanas de vendas.
# "Se eu sei que vou vender 100 unidades nas próximas 4 semanas, quero ter pelo menos 100 unidades em estoque"
# ==================================================
def calcular_alvos(df, todas_semanas, semana_inicio, semanas_alvo):
    """Calcula alvos de estoque para cada SKU/PDV"""
    print("Calculando alvos de estoque...")
    
    if semana_inicio not in todas_semanas:
        raise ValueError(f"Semana inicial {semana_inicio} não encontrada no planejamento de vendas.")
    
    inicio_idx = todas_semanas.index(semana_inicio)
    semanas_simulacao = todas_semanas[inicio_idx: inicio_idx + semanas_alvo]
    if not semanas_simulacao:
        raise ValueError("Lista de semanas para simulação está vazia.")

    for semana in semanas_simulacao:
        idx_semana = todas_semanas.index(semana)
        semanas_somar = todas_semanas[idx_semana: idx_semana + semanas_alvo]
        df[f"ALVO_{semana}"] = df[semanas_somar].sum(axis=1)
    
    return df, semanas_simulacao


In [ ]:
# ==================================================
# 4. FUNÇÃO: SIMULAR REPOSIÇÃO SEMANAL
# ==================================================
def simular_reposicao(df, semanas_simulacao):
    """Simula reposição semana a semana seguindo a lógica de cobertura"""
    print("Simulando reposição...")
    
    coluna_estoque_anterior = "ESTOQUE_INICIAL"
    
    for semana in semanas_simulacao:
        venda_semana = semana
        alvo_semana = f"ALVO_{semana}"
        reposicao_semana = f"REPOSICAO_{semana}"
        estoque_semana = f"ESTOQUE_{semana}"

        base_reposicao = np.maximum(0, df[alvo_semana] - (df[coluna_estoque_anterior] - df[venda_semana]))
        df[reposicao_semana] = np.ceil(base_reposicao).astype(int)

        df[estoque_semana] = np.maximum(0, df[coluna_estoque_anterior] - df[venda_semana] + df[reposicao_semana])
        coluna_estoque_anterior = estoque_semana
    
    return df


In [ ]:
def salvar_resultados(df, semanas_simulacao, pasta_saida):
    print("Salvando resultados...")
    
    print("Colunas disponíveis para salvar:", df.columns.tolist())
    
    colunas_base = [
        "FILIAL",
        "SKU",
        "PRODUTO",
        "COR",
        "TAMANHO",
        "ESTOQUE_ATUAL",
        "CARTEIRA_TOTAL",
        "ESTOQUE_INICIAL",
    ]
    colunas_vendas = semanas_simulacao
    colunas_alvos = [f"ALVO_{s}" for s in semanas_simulacao]
    colunas_reposicao = [f"REPOSICAO_{s}" for s in semanas_simulacao]
    colunas_estoque = [f"ESTOQUE_{s}" for s in semanas_simulacao]
    
    for col in colunas_vendas + colunas_alvos + colunas_reposicao + colunas_estoque:
        if col not in df.columns:
            df[col] = 0
    
    todas_colunas = colunas_base + colunas_vendas + colunas_alvos + colunas_reposicao + colunas_estoque
    colunas_existentes = [col for col in todas_colunas if col in df.columns]
    print("Colunas que serão salvas:", colunas_existentes)
    
    df_final = df[colunas_existentes]
    
    pasta_saida.mkdir(exist_ok=True)
    caminho_saida = pasta_saida / "projecao_sku_pdv.xlsx"
    df_final.to_excel(caminho_saida, index=False)
    
    print(f"[OK] Resultados salvos em: {caminho_saida}")
    return df_final


In [ ]:
# ==================================================
# 5. FUNÇÃO PRINCIPAL
# ==================================================
def main():
    """Executa a simulação completa por SKU/PDV"""
    print("=" * 60)
    print("SIMULAÇÃO DE REPOSIÇÃO POR SKU/PDV")
    print("=" * 60)

    try:
        vendas, todas_semanas = carregar_vendas(ARQUIVO_VENDAS, ABA_VENDAS)
        estoque = carregar_estoque(ARQUIVO_ESTOQUE)
        carteira = carregar_carteira(ARQUIVO_CARTEIRA)

        print()
        print("Combinando dados de vendas, estoque e carteira...")
        dados = pd.merge(vendas, estoque, on=["FILIAL", "SKU"], how="inner")
        dados = pd.merge(dados, carteira, on=["FILIAL", "SKU"], how="left")

        dados["ESTOQUE_ATUAL"] = dados["ESTOQUE_ATUAL"].fillna(0)
        dados["CARTEIRA_TOTAL"] = dados["CARTEIRA_TOTAL"].fillna(0)
        dados["ESTOQUE_INICIAL"] = dados["ESTOQUE_ATUAL"] + dados["CARTEIRA_TOTAL"]

        dados_com_alvos, semanas_simulacao = calcular_alvos(
            dados, todas_semanas, SEMANA_INICIO, SEMANAS_ALVO
        )
        dados_finais = simular_reposicao(dados_com_alvos, semanas_simulacao)
        resultado = salvar_resultados(dados_finais, semanas_simulacao, SAIDA_DIR)

        print("=" * 60)
        print("SIMULAÇÃO CONCLUÍDA COM SUCESSO!")
        print(f"Período: {semanas_simulacao[0]} a {semanas_simulacao[-1]}")
        print(f"SKUs analisados: {len(resultado)}")
        print(f"Filiais: {resultado['FILIAL'].nunique()}")
        print("=" * 60)

    except Exception as err:
        print(f"ERRO: {err}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


🚀 SIMULAÇÃO DE REPOSIÇÃO POR ARTIGO-COR
📊 Carregando vendas por Artigo-Cor...
✅ Vendas: 79084 Artigos-Cor, 33 semanas
📅 TODAS as semanas disponíveis: ['W34_25', 'W35_25', 'W36_25', 'W37_25', 'W38_25', 'W39_25', 'W40_25', 'W41_25', 'W42_25', 'W43_25', 'W44_25', 'W45_25', 'W46_25', 'W47_25', 'W48_25', 'W49_25', 'W50_25', 'W51_25', 'W52_25', 'W01_26', 'W02_26', 'W03_26', 'W04_26', 'W05_26', 'W06_26', 'W07_26', 'W08_26', 'W09_26', 'W10_26', 'W11_26', 'W12_26', 'W13_26', 'W14_26']
🔍 Semana inicial: W34_25
🔍 Posição da semana inicial: 0
🔍 Última semana: W14_26
📊 Carregando vendas por Artigo-Cor...
✅ Vendas: 79084 Artigos-Cor, 33 semanas
📦 Carregando estoque de franquias por Artigo-Cor...
📋 Colunas no estoque: ['FILIAL', 'ArtigoCor', 'Semana_365', 'Estoque Semanal Total']
✅ Estoque: 109988 Artigos-Cor de franquias
📋 Colunas após processamento: ['FILIAL', 'ArtigoCor', 'ESTOQUE_INICIAL']

🔗 Combinando dados...
✅ Merge realizado: 68355 Artigos-Cor de franquias
🎯 Calculando alvos de estoque...
🔄 

C:\Users\gabriel.thiago\AppData\Local\Temp\ipykernel_3056\2878135001.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[estoque_semana] = np.maximum(0, df[coluna_estoque_anterior] - df[venda_semana] + df[reposicao_semana])
C:\Users\gabriel.thiago\AppData\Local\Temp\ipykernel_3056\2878135001.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[reposicao_semana] = np.ceil(base_reposicao * (1 + pct/100.0)).astype(int)
C:\Users\gabriel.thiago\AppData\Local\Temp\ipykernel_3056\2878135001.py:27: PerformanceWarning: DataFrame i

✅ Resultados salvos em: outputs\projecao_artigo_cor.xlsx
🎉 SIMULAÇÃO CONCLUÍDA COM SUCESSO!
📈 Período: W34_25 a W14_26
📊 Artigos-Cor analisados: 68355
🏪 Filiais franquiadas: 570
